<h1>Walmart Sales Forecasting </h1>

## Business Problem

# Problem Statemtent

<b> By using these data we have to Predict the walmart sales forecasting based on different parameters</b>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot

from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import warnings

## Importing Dataset

In [ ]:
train_df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
features_df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
stores_df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

In [ ]:
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.head()

In [ ]:
test_df.columns

In [ ]:
test_df.shape

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

In [ ]:
test_df.isnull()

In [ ]:
test_df.isnull().sum()

In [ ]:
features_df.head()

In [ ]:
features_df.columns

In [ ]:
features_df.shape

In [ ]:
features_df.info()

In [ ]:
features_df.describe()

In [ ]:
features_df.isnull()

In [ ]:
features_df.isnull().sum()

In [ ]:
stores_df.head()

In [ ]:
stores_df.columns

In [ ]:
stores_df.shape

In [ ]:
stores_df.info()

In [ ]:
stores_df.describe()

In [ ]:
stores_df.isnull()

In [ ]:
stores_df.isnull().sum()

In [ ]:
dataset = features_df.merge(stores_df, how= 'inner', on = 'Store')
dataset.head()

In [ ]:
train_df.head()

In [ ]:
dataset.info()

In [ ]:
# Handling The Datetime

from datetime import datetime

dataset['Date'] = pd.to_datetime(dataset['Date'])
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

In [ ]:
dataset['week'] = dataset.Date.dt.isocalendar().week
dataset['year'] = dataset.Date.dt.isocalendar().year

In [ ]:
dataset.head()

In [ ]:
train_df.head()

In [ ]:
# Merge dataset with train_df

train_df = train_df.merge(dataset, how='inner', on=['Store', 'Date', 'IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
# Merge dataset with test_df

test_df = test_df.merge(dataset, how='inner', on=['Store', 'Date', 'IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
def scatter(train_df, column):
    plt.figure()
    plt.scatter(train_df[column], train_df['Weekly_Sales'])
    plt.ylabel('Weekly_Sales')
    plt.xlabel(column)

In [ ]:
scatter(train_df, 'Store')
scatter(train_df, 'Dept')
scatter(train_df, 'IsHoliday')
scatter(train_df, 'Temperature')
scatter(train_df, 'Fuel_Price')
scatter(train_df, 'CPI')
scatter(train_df, 'Unemployment')
scatter(train_df, 'Type')
scatter(train_df, 'Size')

### Correlation Matrix
Let's have a look at the Average Weekly Sales per Year and find out if there is any other holiday peak sales that were not considered by 'IsHoliday' filed

In [ ]:
# Average Weekly Sales for the year 2010
weekly_sales_2010 = train_df[train_df['year'] ==  2010]['Weekly_Sales'].groupby(train_df['week']).mean()
sns.lineplot(weekly_sales_2010.index, weekly_sales_2010.values)

In [ ]:
# Average Weekly Sales for the year 2011
weekly_sales_2011 = train_df[train_df['year']== 2011]['Weekly_Sales'].groupby(train_df['week']).mean()
sns.lineplot(weekly_sales_2011.index, weekly_sales_2011.values)

In [ ]:
# Average Weekly Sales for the year 2012
weekly_sales_2012 = train_df[train_df['year']== 2012]['Weekly_Sales'].groupby(train_df['week']).mean()
sns.lineplot(weekly_sales_2012.index, weekly_sales_2012.values)

In [ ]:
# Plotting the above three plot together

plt.figure(figsize= (20, 10))
sns.lineplot(weekly_sales_2010.index, weekly_sales_2010.values)
sns.lineplot(weekly_sales_2011.index, weekly_sales_2011.values)
sns.lineplot(weekly_sales_2012.index, weekly_sales_2012.values)
plt.grid()
plt.xticks(np.arange(1,60, step= 1))
plt.title('Average Weekly Sales Per Year', fontsize = 20)
plt.xlabel('Week', fontsize = 16)
plt.ylabel('Sales', fontsize = 16)
plt.legend(['2010', '2011', '2012'], loc = 'best', fontsize = 16)
plt.show()

In [ ]:
# Average Sales per Department
weekly_sales = train_df['Weekly_Sales'].groupby(train_df['Dept']).mean()
plt.figure(figsize= (25, 12))
sns.barplot(weekly_sales.index, weekly_sales.values)
plt.grid()
plt.title('Average Weekly Sales Per Department', fontsize = 20)
plt.xlabel('Department', fontsize = 16)
plt.ylabel('Sales', fontsize = 16)
plt.show()

In [ ]:
# Average Sales per Store
weekly_sales = train_df['Weekly_Sales'].groupby(train_df['Store']).mean()
plt.figure(figsize= (25, 12))
sns.barplot(weekly_sales.index, weekly_sales.values)
plt.grid()
plt.title('Average Weekly Sales Per Department', fontsize = 20)
plt.xlabel('Store', fontsize = 16)
plt.ylabel('Sales', fontsize = 16)
plt.show()

In [ ]:
sns.set(style = 'white')
corr = train_df.corr()
mask = np.triu(np.ones_like(corr, dtype = np.bool))
fig, ax = plt.subplots(figsize= (25, 15))
cmap = sns.diverging_palette(220, 10, as_cmap= True)
sns.heatmap(corr, mask = mask, cmap = cmap, vmax = 0.3, center = 0, square = True, linewidth= 0.5, cbar_kws = {'shrink': 0.5}, annot = True)

In [ ]:
# Dropping down the variables that have weak correlation
train_df.drop(columns = ['Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)
test_df.drop(columns = ['Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], inplace = True)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# Splitting Data into Train and Test
X = train_df[['Store', 'Dept', 'IsHoliday','Size','week','year']]
y = train_df['Weekly_Sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
X_train.info()

In [ ]:
# Performing GridSearchCV on Ridge Regression
params = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
ridge_regressor = GridSearchCV(Ridge(), params, cv = 7, scoring = 'neg_mean_absolute_error', n_jobs = -1)
ridge_regressor.fit(X_train, y_train)

In [ ]:
# Predicting train and test results
y_train_pred = ridge_regressor.predict(X_train)
y_test_pred = ridge_regressor.predict(X_test)

In [ ]:
print("Train Results for Ridge Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Ridge Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_test.values, y_test_pred)))
print("R-Squared: ", r2_score(y_test.values, y_test_pred))

In [ ]:
# Performing GridSearchCV on Lasso Regression
params = {'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
lasso_regressor = GridSearchCV(Lasso(), params ,cv = 15,scoring = 'neg_mean_absolute_error', n_jobs = -1)
lasso_regressor.fit(X_train, y_train)

In [ ]:
# Predicting train and test results
y_train_pred = lasso_regressor.predict(X_train)
y_test_pred = lasso_regressor.predict(X_test)

In [ ]:
print("Train Results for Lasso Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Lasso Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_test.values, y_test_pred)))
print("R-Squared: ", r2_score(y_test.values, y_test_pred))

In [ ]:
# Performing GridSearchCV on Decision Tree Regression
from sklearn.tree import DecisionTreeRegressor

depth = list(range(3,30))
param_grid = dict(max_depth = depth)
tree = GridSearchCV(DecisionTreeRegressor(), param_grid, cv = 10)
tree.fit(X_train,y_train)

In [ ]:
# Predicting train and test results
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
print("Train Results for Decision Tree Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Decision Tree Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_test.values, y_test_pred)))
print("R-Squared: ", r2_score(y_test.values, y_test_pred))

In [ ]:
# Performing RandomsearchCV on Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
tuned_params = {'n_estimators': [100, 200], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}  
random_regressor = RandomizedSearchCV(RandomForestRegressor(), tuned_params, n_iter = 3, scoring = 'neg_mean_absolute_error', cv = 3, n_jobs = -1)
random_regressor.fit(X_train, y_train)

In [ ]:
# Predicting train and test results
y_train_pred = random_regressor.predict(X_train)
y_test_pred = random_regressor.predict(X_test)

In [ ]:
print("Train Results for Random Forest Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Random Forest Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_test.values, y_test_pred)))
print("R-Squared: ", r2_score(y_test.values, y_test_pred))

In [ ]:
prediction = lasso_regressor.predict(test_df[['Store', 'Dept', 'IsHoliday','Size','week','year']])
prediction

In [ ]:
df.head()

In [ ]:
df.Weekly_Sales = prediction

In [ ]:
df.to_csv('weekly_sales.csv', index = False)